<a href="https://colab.research.google.com/github/therawther/LPV/blob/main/My_Final_Codes_HPC/Cuda_Matrix_Mul_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/andreinechaev/nvcc4jupyter
!pip install git+file:/content/nvcc4jupyter

Cloning into 'nvcc4jupyter'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (48/48), 8.29 KiB | 848.00 KiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning file:///content/nvcc4jupyter to /tmp/pip-req-build-sl8pu0lf
  Running command git clone --filter=blob:none --quiet file:///content/nvcc4jupyter /tmp/pip-req-build-sl8pu0lf
  Resolved file:///content/nvcc4jupyter to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=d15e8dc0a465a10e4a350c94c1dc9ae52b95f0b2c03d44e26846c50dfb7412a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-3sy8lmvg/wheels/fd/06/13/b58cc0bebf3cd854f673ad262798e45f5ab5a6624b569b745b
Successfully bui

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <cuda_runtime.h>
#include <iostream>
#define N 1000

using namespace std;

__global__ void matrixMul(int *a, int *b, int *c)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < N && col < N)
    {
        int sum = 0;
        for (int k = 0; k < N; k++)
        {
            sum += a[row * N + k] * b[k * N + col];
        }
        c[row * N + col] = sum;
    }
}

int main()
{
    int *a, *b, *c;
    // host matrices
    int *dev_a, *dev_b, *dev_c;
    // device matrices
    int size = N * N * sizeof(int);

    // Allocate memory on host
    a = (int *)malloc(size);
    b = (int *)malloc(size);
    c = (int *)malloc(size);

    // Allocate memory on device
    cudaMalloc((void **)&dev_a, size);
    cudaMalloc((void **)&dev_b, size);
    cudaMalloc((void **)&dev_c, size);

    // Initialize host matrices
    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            a[i * N + j] = i;
            b[i * N + j] = j;
        }
    }

    // Copy host matrices to device
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 1 block and 1024 threads per block
    dim3 blockDim(32, 32);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);
    matrixMul<<<gridDim, blockDim>>>(dev_a, dev_b, dev_c);

    // Copy result from device to host
    cudaMemcpy(c, dev_c, size, cudaMemcpyDeviceToHost);
 
 
    // Print the first 10 values of a
    cout<<" first 10 values of a = ";
    for (int i = 0; i < 10; i++)
    {
        cout << a[i] << " ";
    }
    cout<<endl;
 
    // Print the first 10 values of b
    cout<<" first 10 values of b = ";
    for (int i = 0; i < 10; i++)
    {
        cout << b[i] << " ";
    }
    cout<<endl;

    // Print the first 10 values of the result
    cout<<" first 10 values of the result c = ";
    for (int i = 0; i < 10; i++)
    {
        cout << c[i] << " ";
    }

    // Free memory
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
    free(a);
    free(b);
    free(c);

    return 0;
}

 first 10 values of a = 0 0 0 0 0 0 0 0 0 0 
 first 10 values of b = 0 1 2 3 4 5 6 7 8 9 
 first 10 values of the result c = 0 0 0 0 0 0 0 0 0 0 
